### Лабораторная работа №2
## Тема: Каркасная визуализация выпуклого многогранника. Удаление невидимых линий.
Черных Сергей М8О-305Б-20

Вариант: Правильный октаэдр

Разработать формат представления многогранника и процедуру его каркасной отрисовки в ортографической и изометрической проекциях. Обеспечить удаление невидимых линий и возможность пространственных поворотов и масштабирования многогранника. Обеспечить автоматическое центрирование и изменение размеров изображения при изменении размеров окна.

In [36]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from math import sqrt

vertices = np.array([
        ( 1,  0,  0),
        ( 0,  1,  0),
        ( 0,  0,  1/sqrt(2)),
        (-1,  0,  0),
        (0, -1,  0),
        ( 0,  0, -1/sqrt(2))
    ]
)

faces = np.array([
        (0, 1, 2),
        (1, 3, 2),
        (3, 4, 2),
        (4, 0, 2),
        (0, 5, 1),
        (1, 5, 3),
        (3, 5, 4),
        (4, 5, 0)
    ]
)

def plot_3d(a=1):

    fig = go.Figure(data=[
        go.Mesh3d(
            x=a*vertices[:,0],
            y=a*vertices[:,1],
            z=a*vertices[:,2],

            intensity=np.linalg.norm(vertices, axis=1),
            colorscale=[(0.0, 'rgb(0, 0, 55)'),  (1.0, 'rgb(0, 0, 175)'),],
            i=faces[:,0],
            j=faces[:,1],
            k=faces[:,2],
            name='y',
            showscale=False,
            flatshading=True,
        )
    ])

    fig.update_layout(
        autosize=False,
        width=600,
        height=600,
        margin=dict(
            l=10,
            r=10,
            b=10,
            t=80,
            pad=5
        ),
        paper_bgcolor="silver",
        title_text = "Октаэдр",
    )

    fig.show()

def plot_XOY(a=1):
    x = a * np.array([vertices[0, 0], vertices[1, 0], vertices[3, 0], vertices[4, 0], vertices[0, 0]])
    y = a * np.array([vertices[0, 1], vertices[1, 1], vertices[3, 1], vertices[4, 1], vertices[0, 1]])
    fig = go.Figure(go.Scatter(x=x, y=y, fill="toself", fillcolor = 'rgba(0, 0, 175, 1)'))
    fig.update_layout(
        autosize=False,
        margin=dict(
            l=10,
            r=10,
            b=10,
            t=50,
            pad=1
        ),
        paper_bgcolor="silver",
        title_text='XOY'
    )
    fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
    fig.show()

def plot_XOZ(a=1):
    x = a * np.array([vertices[0, 0], vertices[2, 0], vertices[3, 0], vertices[5, 0], vertices[0, 0]])
    z = a * np.array([vertices[0, 2], vertices[2, 2], vertices[3, 2], vertices[5, 2], vertices[0, 2]])
    fig = go.Figure(go.Scatter(x=x, y=z, fill="toself", fillcolor = 'rgba(0, 0, 175, 1)'))
    fig.update_layout(
        autosize=False,
        margin=dict(
            l=10,
            r=10,
            b=10,
            t=50,
            pad=1
        ),
        paper_bgcolor="silver",
        title_text='XOZ'
    )
    fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
    fig.show()

def plot_YOZ(a=1):
    y = a * np.array([vertices[1, 1], vertices[2, 1], vertices[4, 1], vertices[5, 1], vertices[1, 1]])
    z = a * np.array([vertices[1, 2], vertices[2, 2], vertices[4, 2], vertices[5, 2], vertices[1, 2]])
    fig = go.Figure(go.Scatter(x=y, y=z, fill="toself", fillcolor = 'rgba(0, 0, 175, 1)'))
    fig.update_layout(
        autosize=False,
        margin=dict(
            l=10,
            r=10,
            b=10,
            t=50,
            pad=1
        ),
        paper_bgcolor="silver",
        title_text='YOZ'
    )
    fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
  
    fig.show()
def plot_3dlines(a=1):
    vertices = np.array([
        ( 0,  0,  1/sqrt(2)),
        ( 1,  0,  0),
        ( 0,  0,  1/sqrt(2)),
        ( 0,  1,  0),
        ( 0,  0,  1/sqrt(2)),
        (-1,  0,  0),
        ( 0,  0,  1/sqrt(2)),
        (0, -1,  0),

        ( 1,  0,  0),
        ( 0,  1,  0),
        (-1,  0,  0),
        (0, -1,  0),

        ( 0,  0, -1/sqrt(2)),
        (-1,  0,  0),
        ( 0,  0, -1/sqrt(2)),
        ( 0,  1,  0),
        ( 0,  0, -1/sqrt(2)),
        ( 1,  0,  0),
        ]
    )
    fig = go.Figure(data=go.Scatter3d(
        x=a*vertices[:,0],
        y=a*vertices[:,1],
        z=a*vertices[:,2],
        marker=dict(
            color='LightSkyBlue',
            size=1,
        ),
        line=dict(
                color='darkblue',
                width=2
            )
        )
    )
    fig.update_layout(
        autosize=False,
        width=600,
        height=600,
        margin=dict(
            l=10,
            r=10,
            b=10,
            t=80,
            pad=5
        ),
        paper_bgcolor="silver",
        title_text = "Октаэдр",
    )
    fig.show()
    
def plot_all(a=1):
    plot_3d(a)
    plot_3dlines(a)
    plot_XOY(a)
    plot_XOZ(a)
    plot_YOZ(a)
    plot_3dlines(a)

Введите длину стороны а 

In [2]:
a = 1

In [37]:
plot_all(a)